# **Sentimen Analisis Review Game di Play Store - Evil Nun**

## Import Library

In [80]:
from google_play_scraper import app, reviews, Sort, reviews_all
import pandas as pd

pd.options.mode.chained_assignment = None
import numpy as np

seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/refanzzzz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/refanzzzz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/refanzzzz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Scraping Dataset

In [81]:
# Mengambil semua ulasan dari game dengan ID 'com.keplerians.evilnun' di Google Play Store
# Proses scraping meungkin memerlukan waktu beberapa saat tergantung pada jumlah ulasan yang ada
scraper_view = reviews_all(
    'com.keplerians.evilnun',  # ID aplikasi
    lang='id',  # Bahasa ulasan (default: 'en')
    country='id',  # Negara (default: 'en')
    sort=Sort.MOST_RELEVANT,  # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=12000  # Jumlah maksimum ulasan yang ingin diambil
)

## Loading Dataset

In [82]:
# Membuat DataFrame hasil dari scraping
# Melakukan slicing sebanyak 10000 data, karena reviews_all mengabaikan parameter count dan mengambil seluruh ulasan
game_reviews_df = pd.DataFrame(scraper_view[:12000])

# Mengkonversi DataFrame menjadi berkas CSV
game_reviews_df.to_csv('evil_nun_reviews.csv', index=False)

In [83]:
# Membuat DataFrame dari hasil scrape_view
game_reviews_df = pd.DataFrame(scraper_view[:12000])

# Menghitung jumlah baris dan kolom pada DataFrame
reviews_count, columns_count = game_reviews_df.shape

print("Jumlah ulasan: ", reviews_count)
print("Jumlah kolom: ", columns_count)

Jumlah ulasan:  12000
Jumlah kolom:  11


In [84]:
# Menampilkan lima baris pertama dari DataFrame game_reviews_df
game_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,55ed02a9-0ed3-4012-9cff-3ccd6a558172,Alif Fathan,https://play-lh.googleusercontent.com/a/ACg8oc...,Terima kasih untuk Keplerians/Developer yg tel...,5,10,1.9.1,2025-04-09 15:56:37,None,NaT,1.9.1
1,4e4b81f5-7e49-4c20-8b59-7e8d05546b92,Maura Qory Muzdalifa,https://play-lh.googleusercontent.com/a-/ALV-U...,"jujur game ini seru bgt😖🙏, tapi masalah nya in...",5,41,1.9.1,2025-03-23 13:52:19,None,NaT,1.9.1
2,cb2d5f74-8a08-4c84-8bd0-172e9ad2d721,eko setiawan,https://play-lh.googleusercontent.com/a/ACg8oc...,Dari 2019 sampai 2025 (1.8.9) game ini udah ba...,2,37,1.9.0,2025-01-31 17:01:13,Hi! This issue has been just fixed 🔧 You can u...,2025-03-25 18:30:55,1.9.0
3,e331edd9-69d5-41b3-bdfc-3781dfd139b5,Jhosu Tumanggor,https://play-lh.googleusercontent.com/a-/ALV-U...,"ngebug nya banyak,pas topeng nya sudah terkump...",3,1,1.9.1,2025-03-18 21:22:20,None,NaT,1.9.1
4,df9b4228-d28a-4df5-a02c-9316a7c32195,Keisya Kamilla,https://play-lh.googleusercontent.com/a/ACg8oc...,Terima kasih kepada developer Keplerians karen...,5,14,1.9.1,2025-03-21 20:29:49,Hi! This issue has been just fixed 🔧 You can u...,2025-03-18 20:49:27,1.9.1


## Cleaning Dataset

In [85]:
# Menampilkan informasi tentang DataFrame game_reviews_df
game_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              12000 non-null  object        
 1   userName              12000 non-null  object        
 2   userImage             12000 non-null  object        
 3   content               12000 non-null  object        
 4   score                 12000 non-null  int64         
 5   thumbsUpCount         12000 non-null  int64         
 6   reviewCreatedVersion  4298 non-null   object        
 7   at                    12000 non-null  datetime64[ns]
 8   replyContent          253 non-null    object        
 9   repliedAt             253 non-null    datetime64[ns]
 10  appVersion            4298 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 1.0+ MB


In [86]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari game_reviews_df
clean_df = game_reviews_df.dropna()

# Menampilkan informasi tentang DataFrame clean_df
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 2 to 11953
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              116 non-null    object        
 1   userName              116 non-null    object        
 2   userImage             116 non-null    object        
 3   content               116 non-null    object        
 4   score                 116 non-null    int64         
 5   thumbsUpCount         116 non-null    int64         
 6   reviewCreatedVersion  116 non-null    object        
 7   at                    116 non-null    datetime64[ns]
 8   replyContent          116 non-null    object        
 9   repliedAt             116 non-null    datetime64[ns]
 10  appVersion            116 non-null    object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 10.9+ KB


In [87]:
# Mengecek apakah ada data yang duplikat
duplicate_count = clean_df.duplicated().sum()
print("Jumlah duplikasi: ", duplicate_count)

Jumlah duplikasi:  0


## Preprocessing Text

In [88]:
# Membuat helper function untuk membersihkan teks
def cleaning_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus hashtag
    text = re.sub(r'RT[\s]+', '', text)  # Menghapus retweet
    text = re.sub(r'http\S+', '', text)  # Menghapus link
    text = re.sub(r'[0-9]+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ')  # Mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
    text = text.strip(' ')  # Menghapus karakter awal dan akhir

    return text

In [89]:
# Membuat helper function untuk mengubah teks menjadi huruf kecil
def casefolding_text(text):
    text = text.lower()
    return text

In [90]:
# Memecah atau membagi string, teks menjadi daftar token
def tokenizing_text(text):
    text = word_tokenize(text)
    return text

In [91]:
# Menghapus stopwords dalam teks
def filtering_text(text):
    list_stopwords = set(stopwords.words('indonesian'))
    list_stopwords1 = set(stopwords.words('english'))
    list_stopwords.update(list_stopwords1)
    list_stopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy", "anjir", "njir"])

    filtered = []

    for txt in text:
        if txt not in list_stopwords:
            filtered.append(txt)

    text = filtered
    return text

In [92]:
# Membuat helper function untuk mengubah kata menjadi kata dasar
def stemming_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [93]:
# Mengkonversi list kata menjadi kalimat
def to_sentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [94]:
# Menghapus angka pada teks
def remove_number(text):
    text_without_number = ''.join([char for char in text if not char.isdigit()])
    return text_without_number


In [95]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar",
              "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia",
              "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber",
              "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja",
              "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku",
              "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing",
              "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya",
              "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam",
              "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu",
              "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan",
              "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur",
              "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan",
              "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan",
              "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana",
              "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh",
              "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah",
              "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat",
              "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa",
              "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur",
              "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan",
              "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi",
              "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut",
              "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja",
              "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep",
              "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi",
              "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek",
              "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor",
              "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh",
              "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan",
              "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana",
              "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan",
              "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana",
              "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian",
              "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya",
              "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile",
              "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi",
              "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa",
              "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja",
              "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas",
              "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana",
              "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini",
              "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya",
              "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok",
              "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan",
              "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong",
              "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah",
              "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion",
              "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi",
              "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga",
              "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar",
              "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur",
              "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan",
              "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah",
              "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya",
              "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional",
              "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau",
              "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat",
              "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau",
              "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus",
              "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin",
              "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat",
              "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu",
              "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah",
              "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan",
              "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi",
              "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan",
              "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa",
              "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu",
              "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih",
              "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa",
              "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan",
              "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi",
              "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau",
              "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman",
              "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional",
              "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan",
              "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana",
              "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ",
              "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak",
              "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta",
              "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil",
              "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan",
              "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim",
              "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok",
              "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan",
              "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini",
              "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng",
              "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi",
              "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu",
              "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic",
              "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah",
              "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala",
              "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada",
              "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah",
              "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani",
              "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang",
              "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin",
              "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren",
              "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti",
              "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi",
              "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi",
              "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah",
              "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya",
              "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa",
              "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya",
              "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip",
              "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat",
              "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik",
              "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit",
              "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat",
              "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf",
              "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti",
              "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit",
              "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi",
              "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir",
              "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu",
              "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne",
              "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman",
              "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai",
              "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih",
              "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu",
              "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah",
              "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu",
              "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur",
              "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat",
              "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya",
              "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya",
              "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord",
              "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja",
              "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja",
              "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa",
              "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin",
              "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik",
              "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar",
              "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas",
              "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden",
              "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah",
              "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar",
              "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis",
              "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur",
              "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli",
              "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak",
              "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh",
              "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa",
              "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan",
              "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan",
              "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep",
              "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek",
              "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek",
              "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck",
              "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut",
              "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu",
              "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu",
              "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa",
              "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo",
              "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus",
              "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh",
              "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali",
              "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah",
              "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi",
              "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji",
              "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian",
              "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak",
              "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten",
              "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar",
              "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti",
              "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan",
              "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan",
              "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian",
              "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus",
              "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer",
              "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada",
              "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya",
              "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan",
              "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur",
              "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat",
              "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh",
              "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu",
              "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih",
              "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main",
              "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan",
              "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus",
              "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras",
              "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede",
              "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing",
              "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik",
              "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf",
              "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik",
              "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut",
              "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa",
              "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa",
              "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng",
              "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat",
              "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut",
              "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan",
              "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari",
              "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok",
              "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas",
              "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi",
              "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal",
              "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya",
              "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya",
              "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa",
              "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu",
              "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon",
              "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal",
              "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang",
              "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman",
              "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi",
              "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya",
              "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal",
              "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik",
              "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras",
              "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat",
              "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama",
              "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu",
              "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang",
              "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut",
              "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu",
              "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang",
              "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah",
              "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak",
              "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi",
              "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}

In [96]:
# Mengubah kata slang menjadi formal
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [97]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaning_text)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di dalam kolom 'text_casefolding'
clean_df['text_casefolding'] = clean_df['text_clean'].apply(casefolding_text)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefolding'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di kolom 'text_tokenizing'
clean_df['text_tokenizing'] = clean_df['text_slangwords'].apply(tokenizing_text)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopwords'] = clean_df['text_tokenizing'].apply(filtering_text)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di kolom 'text_to_sentence'
clean_df['text_final'] = clean_df['text_stopwords'].apply(to_sentence)

In [98]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefolding,text_slangwords,text_tokenizing,text_stopwords,text_final
2,cb2d5f74-8a08-4c84-8bd0-172e9ad2d721,eko setiawan,https://play-lh.googleusercontent.com/a/ACg8oc...,Dari 2019 sampai 2025 (1.8.9) game ini udah ba...,2,37,1.9.0,2025-01-31 17:01:13,Hi! This issue has been just fixed 🔧 You can u...,2025-03-25 18:30:55,1.9.0,Dari sampai game ini udah bagus banget tapi...,dari sampai game ini udah bagus banget tapi...,dari sampai game ini sudah bagus banget tapi k...,"[dari, sampai, game, ini, sudah, bagus, banget...","[game, bagus, banget, gamenya, stuck, loading,...",game bagus banget gamenya stuck loading scene ...
4,df9b4228-d28a-4df5-a02c-9316a7c32195,Keisya Kamilla,https://play-lh.googleusercontent.com/a/ACg8oc...,Terima kasih kepada developer Keplerians karen...,5,14,1.9.1,2025-03-21 20:29:49,Hi! This issue has been just fixed 🔧 You can u...,2025-03-18 20:49:27,1.9.1,Terima kasih kepada developer Keplerians karen...,terima kasih kepada developer keplerians karen...,terima kasih kepada developer keplerians karen...,"[terima, kasih, kepada, developer, keplerians,...","[terima, kasih, developer, keplerians, memperb...",terima kasih developer keplerians memperbaiki ...
5,1a29619b-dd88-4830-9367-f94731b9908b,dawai hidayat,https://play-lh.googleusercontent.com/a-/ALV-U...,Diversi 1.9.0 gamenya wajib ad iklan baru bisa...,2,13,1.9.0,2025-01-31 16:16:18,Hi! This issue has been just fixed 🔧 You can u...,2025-03-25 18:30:59,1.9.0,Diversi gamenya wajib ad iklan baru bisa masu...,diversi gamenya wajib ad iklan baru bisa masu...,diversi gamenya wajib ada iklan baru bisa masu...,"[diversi, gamenya, wajib, ada, iklan, baru, bi...","[diversi, gamenya, wajib, iklan, masuk, pakai,...",diversi gamenya wajib iklan masuk pakai koneks...
10,a8763dff-67cf-43b0-af1c-aba0f23dc53e,Puas anes Anes,https://play-lh.googleusercontent.com/a/ACg8oc...,"""Game makin rusak setelah update! Dulu lancar,...",1,8,1.9.0,2025-03-02 17:32:37,Hi! This issue has been just fixed 🔧 You can u...,2025-03-24 21:48:07,1.9.0,Game makin rusak setelah update Dulu lancar se...,game makin rusak setelah update dulu lancar se...,game makin rusak setelah update dulu lancar se...,"[game, makin, rusak, setelah, update, dulu, la...","[game, rusak, update, lancar, stuck, loading, ...",game rusak update lancar stuck loading coba ha...
11,83f17e8f-fe49-4fe5-8538-b22a12d0d06f,Izma aulida,https://play-lh.googleusercontent.com/a-/ALV-U...,"Tolong dong, item/barang nya jangan di bikin b...",2,4,1.9.0,2025-02-03 10:53:27,Hi! This issue has been just fixed 🔧 You can u...,2025-03-25 18:28:49,1.9.0,Tolong dong itembarang nya jangan di bikin bug...,tolong dong itembarang nya jangan di bikin bug...,tolong dong itembarang nya jangan di bikin bug...,"[tolong, dong, itembarang, nya, jangan, di, bi...","[tolong, itembarang, bikin, bug, udh, misi, bi...",tolong itembarang bikin bug udh misi bikin bon...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,43405328-ccf2-4899-987d-17c356ac5bd4,wibu sejati anime,https://play-lh.googleusercontent.com/a-/ALV-U...,Gamenya mantep seru!!!!!!!!!!!!!,5,0,1.7.2,2021-09-05 17:04:26,Thank you! :),2020-07-08 16:58:57,1.7.2,Gamenya mantep seru,gamenya mantep seru,gamenya mantap seru,"[gamenya, mantap, seru]","[gamenya, mantap, seru]",gamenya mantap seru
11578,01938281-81c9-4066-85a4-778a29a594ff,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,gak masuk masuk uang nya,1,1,1.7.0,2019-08-20 15:21:27,Hi! Today will be updated with a lot of fixes....,2019-08-20 17:51:30,1.7.0,gak masuk masuk uang nya,gak masuk masuk uang nya,gak masuk masuk uang nya,"[gak, masuk, masuk, uang, nya]","[masuk, masuk, uang]",masuk masuk uang
11579,a96157b8-3ece-4066-8a28-e06394d95a22,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Permainannya sangat menegangkan,5,0,1.6.2,2019-11-04 21:06:42,Glad y